In [12]:
import tensorflow as tf
print(tf.__version__)

2.9.1


In [13]:
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:
import pandas as pd
#pd.options.mode.chained_assignment = None
import numpy as np
from scipy import stats

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.applications import *

In [16]:
import sys
sys.path.append('../utils')  # Add the 'utils' folder to the Python path

from classification_cross_validation import pipeline_cross_val  # Import function from helper.py

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.svm import SVC

In [65]:
import os

DATA_FOLDER = 'C:\\Users\\hayadi\\Desktop\\Smoking Vocal Biomarkers\\VB Smoking Project\\data'
EXTRACTION = "EXTRACT_2023_03_16_12_09_45"

EMBEDDINGS_FOLDER = '..\\embeddings\\for model training'

### Results for English-speaking female participants

##### Data reading 
(explicit participant data is private, the embeddings extracted from the audio recordings   are however available)

In [66]:
df = pd.read_csv(os.path.join(DATA_FOLDER, EXTRACTION, "female_english_only_matched_sg_cg_16032023_manual_approach.csv"))

In [79]:
df[['surveyLocale', 'age', 'gender', 'weight', 'weight_loss', 'height', 'educ', 'ethnicity', 'blood', 'smk_1', 'smk_4', 'headache', 'sore-throat', ]]

,surveyLocale,age,gender,weight,weight_loss,height,educ,ethnicity,blood,smk_1,smk_2,smk_3,smk_4,headache,sore-throat
0,1,55.0,female,86.0,no,165.0,highschool,white,B-,daily,NaN,NaN,8.0,yes,0
1,1,45.0,female,113.4,no,160.0,highschool,white,unknown,daily,NaN,NaN,20.0,yes,0
2,1,42.0,female,138.0,no,162.0,bachelor,white,A+,daily,NaN,NaN,18.0,yes,0
3,1,45.0,female,120.0,no,162.0,master,black,unknown,daily,NaN,NaN,10.0,no,0
4,1,61.0,female,83.0,no,177.0,master,mixed,A+,daily,NaN,NaN,6.0,yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,1,55.0,female,NaN,no,NaN,bachelor,white,O+,not-at-all,NaN,not-at-all,NaN,no,1
354,1,50.0,female,NaN,no,NaN,highschool,white,A+,not-at-all,NaN,not-at-all,NaN,yes,0
355,1,20.0,female,NaN,no,NaN,highschool,black,unknown,not-at-all,NaN,not-at-all,NaN,no,1
356,1,66.0,female,NaN,no,NaN,master,white,A-,not-at-all,NaN,not-at-all,NaN,no,0


In [76]:
for col in df.drop(columns=['Unnamed: 0', 'index', 'Id', 'UniqueId', 'smk_4_0-9', 'smk_4_10+', 'ps', 'matched', 'prolific.studyId.value']):
    print(col)

mother-tongue
age
gender
weight
weight_loss
height
educ
ethnicity
blood
smk_1
smk_3
alc_1
pain_1
chestpain
bowel_mov
fever_1
headache
sore-throat
balance
parrot_1
stress
fss_1
fss_2
fss_3
fss_4
fss_5
fss_6
fss_7
fss_8
fss_9
fss_score
qol_1
qol_2
qol_3
qol_4
qol_5
phq9_1
phq9_2
phq9_3
phq9_4
phq9_5
phq9_6
phq9_7
phq9_8
phq9_9
scs_1
scs_2
rqol_1
rqol_2
rqol_3
rqol_4
rqol_5
rqol_6
rqol_7
rqol_8
rqol_9
rqol_10
rqol_11
rbdsq_1
rbdsq_2
rbdsq_3
rbdsq_4
rbdsq_5
rbdsq_6_1
rbdsq_6_2
rbdsq_6_3
rbdsq_6_4
rbdsq_7
rbdsq_8
rbdsq_9
rbdsq_10
drug_1
hormon_1
cvddis
endodis
immaldis_1
pulmdis
cancer
comdis
metaboldis
gastrodis
neurodis
otherdis
psychodis
alc_2
alc_3
hormon_2
smk_4
pain_drugs
choles_1
diab_1
hypertens_1
anx_1
antireflux_1
thyroid_1
throat_1
cortico_1
antibio_1
relaxmusc_1
decongest_1
diuret_1
antidiarr_1
antihis_1
antichol_1
vitc_1
anticoag_1
typediabetes
agediabetes
tablet
insulin-1
insulin-2
hba1c
cgm-1
hist-diabetes
paid-1
paid-2
paid-3
paid-4
paid-5
paid-6
paid-7
paid-8
paid-9
paid-10

In [68]:
for col in df.columns:
    print(col)

Unnamed: 0
index
Id
UniqueId
mother-tongue
age
gender
weight
weight_loss
height
educ
ethnicity
blood
smk_1
smk_3
alc_1
pain_1
chestpain
bowel_mov
fever_1
headache
sore-throat
balance
parrot_1
stress
fss_1
fss_2
fss_3
fss_4
fss_5
fss_6
fss_7
fss_8
fss_9
fss_score
qol_1
qol_2
qol_3
qol_4
qol_5
phq9_1
phq9_2
phq9_3
phq9_4
phq9_5
phq9_6
phq9_7
phq9_8
phq9_9
scs_1
scs_2
rqol_1
rqol_2
rqol_3
rqol_4
rqol_5
rqol_6
rqol_7
rqol_8
rqol_9
rqol_10
rqol_11
rbdsq_1
rbdsq_2
rbdsq_3
rbdsq_4
rbdsq_5
rbdsq_6_1
rbdsq_6_2
rbdsq_6_3
rbdsq_6_4
rbdsq_7
rbdsq_8
rbdsq_9
rbdsq_10
drug_1
hormon_1
cvddis
endodis
immaldis_1
pulmdis
cancer
comdis
metaboldis
gastrodis
neurodis
otherdis
psychodis
alc_2
alc_3
hormon_2
smk_4
pain_drugs
choles_1
diab_1
hypertens_1
anx_1
antireflux_1
thyroid_1
throat_1
cortico_1
antibio_1
relaxmusc_1
decongest_1
diuret_1
antidiarr_1
antihis_1
antichol_1
vitc_1
anticoag_1
typediabetes
agediabetes
tablet
insulin-1
insulin-2
hba1c
cgm-1
hist-diabetes
paid-1
paid-2
paid-3
paid-4
paid-5
paid-6

In [67]:
df['currently_smoking'].value_counts()

1    179
0    179
Name: currently_smoking, dtype: int64

In [35]:
y_smk = df['currently_smoking'].values
y_smk.shape

(358,)

##### Features

In [36]:
reading_embeddings = np.load(os.path.join(EMBEDDINGS_FOLDER, 'compare_reading_embeddings_female_english.npy'))
counting_embeddings = np.load(os.path.join(EMBEDDINGS_FOLDER, 'vggish_counting_embeddings_female_english.npy'), allow_pickle = True)
a_vowel_phonation_embeddings = np.load(os.path.join(EMBEDDINGS_FOLDER, 'byola_2048_a_vowel_phonation_embeddings_female_english.npy'))

In [37]:
RANDOM_STATE = 42
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
scaler = StandardScaler()

In [38]:
N_FEATURES = 100
N_STEPS = 100

#### Reading

In [39]:
embeddings_var = reading_embeddings
embeddings_var.shape

(358, 6374)

##### Grid search with feature selection

In [40]:
search_space = { 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [3, 5, 7],
                 'classifier__weights': ['uniform', 'distance'],
                 }


model = Pipeline([('scaler', scaler), ('reduction', RFE(RandomForestClassifier(random_state=42), n_features_to_select=N_FEATURES, step=N_STEPS)), ('classifier', KNeighborsClassifier())])                 

clf = GridSearchCV(model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

knn_neighbours = clf.best_params_['classifier__n_neighbors']
knn_weights = clf.best_params_['classifier__weights']

print(clf.best_params_)

{'classifier': KNeighborsClassifier(n_neighbors=7, weights='distance'), 'classifier__n_neighbors': 7, 'classifier__weights': 'distance'}


In [41]:
search_space = {'classifier': [LogisticRegression(solver='liblinear')],
                'classifier__C': [0.01, 0.1, 1.0, 10]}

model = Pipeline([('scaler', scaler), ('reduction', RFE(RandomForestClassifier(random_state=42), n_features_to_select=N_FEATURES, step=N_STEPS)), ('classifier', LogisticRegression())])                 

clf = GridSearchCV(model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

lr_c = clf.best_params_['classifier__C']

print(print(clf.best_params_))

{'classifier': LogisticRegression(C=0.01, solver='liblinear'), 'classifier__C': 0.01}
None


In [42]:
search_space = {'classifier': [ExtraTreesClassifier(random_state=RANDOM_STATE)],
                'classifier__n_estimators': [10, 30, 50, 100, 200],
                'classifier__max_depth': [8, 10, 12, 20]}

model = Pipeline([('scaler', scaler), ('reduction', RFE(RandomForestClassifier(random_state=42), n_features_to_select=N_FEATURES, step=N_STEPS)), ('classifier', ExtraTreesClassifier())])                 

clf = GridSearchCV(model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

etc_max_depth = clf.best_params_['classifier__max_depth']
etc_n_estimators = clf.best_params_['classifier__n_estimators']

print(clf.best_params_)

{'classifier': ExtraTreesClassifier(max_depth=10, n_estimators=30, random_state=42), 'classifier__max_depth': 10, 'classifier__n_estimators': 30}


In [43]:
search_space = {'classifier': [RandomForestClassifier(max_depth=4, max_leaf_nodes=5, random_state=RANDOM_STATE)],
                'classifier__max_depth': [5, 10, None],
                'classifier__n_estimators': [10, 20, 30, 40, 50, 100]}

model = Pipeline([('scaler', scaler), ('reduction', RFE(RandomForestClassifier(random_state=42), n_features_to_select=N_FEATURES, step=N_STEPS)), ('classifier', RandomForestClassifier())])                 

clf = GridSearchCV(model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

rf_max_depth = clf.best_params_['classifier__max_depth']
rf_n_estimators = clf.best_params_['classifier__n_estimators']

print(clf.best_params_)

{'classifier': RandomForestClassifier(max_depth=5, max_leaf_nodes=5, random_state=42), 'classifier__max_depth': 5, 'classifier__n_estimators': 100}


In [44]:
search_space = {'classifier': [LinearDiscriminantAnalysis()],
                'classifier__solver': ['svd', 'lsqr']}

model = Pipeline([('scaler', scaler), ('reduction', RFE(RandomForestClassifier(random_state=42), n_features_to_select=N_FEATURES, step=N_STEPS)), ('classifier', LinearDiscriminantAnalysis())])              

clf = GridSearchCV(model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

lda_solver = clf.best_params_['classifier__solver']

print(clf.best_params_)


{'classifier': LinearDiscriminantAnalysis(), 'classifier__solver': 'svd'}


In [45]:
search_space = { 'classifier': [MLPClassifier(random_state=RANDOM_STATE, max_iter = 1000)],
                 'classifier__hidden_layer_sizes': [(4,), (8,), (20,), (50,), (32,), (32,6,), (100,), (256, 128,)],
                 'classifier__activation': ['relu', 'tanh', 'logistic'],
                 'classifier__solver': ['adam', 'lbfgs']}

model = Pipeline([('scaler', scaler), ('reduction', RFE(RandomForestClassifier(random_state=42), n_features_to_select=N_FEATURES, step=N_STEPS)), ('classifier', MLPClassifier(random_state=1, max_iter=1000))])               

clf = GridSearchCV(model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

mlp_hidden_layer = clf.best_params_['classifier__hidden_layer_sizes']
mlp_activation = clf.best_params_['classifier__activation']
mlp_solver = clf.best_params_['classifier__solver']

print(clf.best_params_)

{'classifier': MLPClassifier(hidden_layer_sizes=(32,), max_iter=1000, random_state=42,
              solver='lbfgs'), 'classifier__activation': 'relu', 'classifier__hidden_layer_sizes': (32,), 'classifier__solver': 'lbfgs'}


In [46]:
search_space = {'classifier': [SVC(kernel='rbf')],
                'classifier__C': [1e4, 1e5, 1e6],
                'classifier__gamma': [1e-6, 1e-5, 1e-4, 1e-3]}

model = Pipeline([('scaler', scaler), ('reduction', RFE(RandomForestClassifier(random_state=42), n_features_to_select=N_FEATURES, step=N_STEPS)), ('classifier', SVC(kernel='rbf'))])              

clf = GridSearchCV(model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

svc_c = clf.best_params_['classifier__C']
svc_g = clf.best_params_['classifier__gamma']


print(clf.best_params_)

{'classifier': SVC(C=10000.0, gamma=1e-06), 'classifier__C': 10000.0, 'classifier__gamma': 1e-06}


In [47]:
search_space = {'classifier': [SVC(kernel='sigmoid')],
                'classifier__C': [1e4, 1e5, 1e6],
                'classifier__gamma': [1e-6, 1e-5, 1e-4, 1e-3]}

model = Pipeline([('scaler', scaler), ('reduction', RFE(RandomForestClassifier(random_state=42), n_features_to_select=N_FEATURES, step=N_STEPS)), ('classifier', SVC(kernel='sigmoid'))])                 

clf = GridSearchCV(model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

svc_sig_c = clf.best_params_['classifier__C']
svc_sig_g = clf.best_params_['classifier__gamma']

print(clf.best_params_)

{'classifier': SVC(C=10000.0, gamma=1e-06, kernel='sigmoid'), 'classifier__C': 10000.0, 'classifier__gamma': 1e-06}


##### Cross-validation

In [48]:
# Model building to train
names = ['K Nearest Neighbors','Logistic Regression', 'ExtraTrees Classifier','Random Forest','LDA' ,'MLP Classifier' ,'SVM rbf', 'SVM sigmoid']
Classifiers = [
               KNeighborsClassifier(n_neighbors=knn_neighbours, weights=knn_weights),
               LogisticRegression(solver='liblinear', C=lr_c), 
               ExtraTreesClassifier(random_state= RANDOM_STATE, max_depth=etc_max_depth, n_estimators=etc_n_estimators),
               RandomForestClassifier(max_leaf_nodes=5, random_state= RANDOM_STATE, max_depth=rf_max_depth, n_estimators=rf_n_estimators), 
               LinearDiscriminantAnalysis(solver = lda_solver),
               MLPClassifier(random_state= RANDOM_STATE, hidden_layer_sizes=mlp_hidden_layer, activation=mlp_activation, solver=mlp_solver, max_iter=1000),
               svm.SVC(kernel='rbf', C=svc_c, gamma=svc_g),
               svm.SVC(kernel='sigmoid', C=svc_sig_c, gamma=svc_sig_g)]

In [49]:
pipeline_cross_val(names, Classifiers, embeddings_var, y_smk, reduction = 'Feature_selection', n_features_to_select=N_FEATURES, step=N_STEPS, kfold=kfold)

,Classifier,accuracy,balanced_accuracy,Precision,Recall,F1,AUC
0,K Nearest Neighbors,0.65 (0.04),0.65 (0.04),0.68 (0.07),0.60 (0.06),0.63 (0.03),0.70 (0.05)
1,Logistic Regression,0.64 (0.05),0.64 (0.05),0.65 (0.06),0.64 (0.07),0.64 (0.05),0.73 (0.04)
2,ExtraTrees Classifier,0.71 (0.04),0.71 (0.04),0.71 (0.06),0.73 (0.07),0.72 (0.04),0.77 (0.07)
3,Random Forest,0.72 (0.07),0.72 (0.07),0.72 (0.08),0.72 (0.06),0.72 (0.06),0.76 (0.08)
4,LDA,0.61 (0.07),0.61 (0.07),0.62 (0.07),0.58 (0.10),0.60 (0.08),0.66 (0.06)
5,MLP Classifier,0.66 (0.05),0.66 (0.05),0.65 (0.05),0.68 (0.07),0.66 (0.06),0.70 (0.05)
6,SVM rbf,0.66 (0.02),0.66 (0.02),0.67 (0.05),0.65 (0.08),0.66 (0.03),0.70 (0.04)
7,SVM sigmoid,0.68 (0.04),0.68 (0.04),0.68 (0.05),0.67 (0.06),0.67 (0.03),0.72 (0.04)


#### Counting

In [56]:
embeddings_var = counting_embeddings
embeddings_var.shape

(358, 128)

##### Grid search without PCA 

In [57]:
search_space = { 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [3, 5, 7],
                 'classifier__weights': ['uniform', 'distance'],
                 }

pca_model =  Pipeline([('scaler', scaler), ('classifier', KNeighborsClassifier())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

knn_neighbours = clf.best_params_['classifier__n_neighbors']
knn_weights = clf.best_params_['classifier__weights']

print(clf.best_params_)

{'classifier': KNeighborsClassifier(n_neighbors=7, weights='distance'), 'classifier__n_neighbors': 7, 'classifier__weights': 'distance'}


In [58]:
search_space = {'classifier': [LogisticRegression(solver='liblinear')],
                'classifier__C': [0.01, 0.1, 1.0, 10]}

pca_model =  Pipeline([('scaler', scaler), ('classifier', LogisticRegression())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

lr_c = clf.best_params_['classifier__C']

print(print(clf.best_params_))

{'classifier': LogisticRegression(C=0.01, solver='liblinear'), 'classifier__C': 0.01}
None


In [59]:
search_space = {'classifier': [ExtraTreesClassifier(random_state=RANDOM_STATE)],
                'classifier__n_estimators': [10, 30, 50, 100, 200],
                'classifier__max_depth': [8, 10, 12, 20]}

pca_model =  Pipeline([('scaler', scaler), ('classifier', ExtraTreesClassifier())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

etc_max_depth = clf.best_params_['classifier__max_depth']
etc_n_estimators = clf.best_params_['classifier__n_estimators']

print(clf.best_params_)

{'classifier': ExtraTreesClassifier(max_depth=12, n_estimators=200, random_state=42), 'classifier__max_depth': 12, 'classifier__n_estimators': 200}


In [60]:
search_space = {'classifier': [RandomForestClassifier(max_depth=4, max_leaf_nodes=5, random_state=RANDOM_STATE)],
                'classifier__max_depth': [5, 10, None],
                'classifier__n_estimators': [10, 20, 30, 40, 50, 100]}

pca_model =  Pipeline([('scaler', scaler), ('classifier', RandomForestClassifier())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

rf_max_depth = clf.best_params_['classifier__max_depth']
rf_n_estimators = clf.best_params_['classifier__n_estimators']

print(clf.best_params_)

{'classifier': RandomForestClassifier(max_depth=5, max_leaf_nodes=5, n_estimators=20,
                       random_state=42), 'classifier__max_depth': 5, 'classifier__n_estimators': 20}


In [61]:
search_space = {'classifier': [LinearDiscriminantAnalysis()],
                'classifier__solver': ['svd', 'lsqr']}

pca_model =  Pipeline([('scaler', scaler), ('classifier', LinearDiscriminantAnalysis())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

lda_solver = clf.best_params_['classifier__solver']

print(clf.best_params_)


{'classifier': LinearDiscriminantAnalysis(), 'classifier__solver': 'svd'}


In [62]:
search_space = { 'classifier': [MLPClassifier(random_state=RANDOM_STATE, max_iter = 1000)],
                 'classifier__hidden_layer_sizes': [(4,), (8,), (20,), (50,), (32,), (32,6,), (100,), (256, 128,)],
                 'classifier__activation': ['relu', 'tanh', 'logistic'],
                 'classifier__solver': ['adam', 'lbfgs']}

pca_model =  Pipeline([('scaler', scaler), ('classifier',MLPClassifier(random_state=1, max_iter=1000))])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

mlp_hidden_layer = clf.best_params_['classifier__hidden_layer_sizes']
mlp_activation = clf.best_params_['classifier__activation']
mlp_solver = clf.best_params_['classifier__solver']

print(clf.best_params_)

{'classifier': MLPClassifier(activation='tanh', hidden_layer_sizes=(50,), max_iter=1000,
              random_state=42), 'classifier__activation': 'tanh', 'classifier__hidden_layer_sizes': (50,), 'classifier__solver': 'adam'}


In [63]:
search_space = {'classifier': [SVC(kernel='rbf')],
                'classifier__C': [1e4, 1e5, 1e6],
                'classifier__gamma': [1e-6, 1e-5, 1e-4, 1e-3]}

pca_model =  Pipeline([('scaler', scaler), ('classifier', ('classifier', SVC(kernel='rbf')))])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

svc_c = clf.best_params_['classifier__C']
svc_g = clf.best_params_['classifier__gamma']


print(clf.best_params_)

{'classifier': SVC(C=10000.0, gamma=1e-06), 'classifier__C': 10000.0, 'classifier__gamma': 1e-06}


In [64]:
search_space = {'classifier': [SVC(kernel='sigmoid')],
                'classifier__C': [1e4, 1e5, 1e6],
                'classifier__gamma': [1e-6, 1e-5, 1e-4, 1e-3]}

pca_model =  Pipeline([('scaler', scaler), ('classifier', SVC(kernel='sigmoid'))])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

svc_sig_c = clf.best_params_['classifier__C']
svc_sig_g = clf.best_params_['classifier__gamma']

print(clf.best_params_)

{'classifier': SVC(C=10000.0, gamma=1e-06, kernel='sigmoid'), 'classifier__C': 10000.0, 'classifier__gamma': 1e-06}


##### Cross-validation

In [65]:
# Model building to train
names = ['K Nearest Neighbors','Logistic Regression', 'ExtraTrees Classifier','Random Forest','LDA' ,'MLP Classifier' ,'SVM rbf', 'SVM sigmoid']
Classifiers = [
               KNeighborsClassifier(n_neighbors=knn_neighbours, weights=knn_weights),
               LogisticRegression(solver='liblinear', C=lr_c), 
               ExtraTreesClassifier(random_state= RANDOM_STATE, max_depth=etc_max_depth, n_estimators=etc_n_estimators),
               RandomForestClassifier(max_leaf_nodes=5, random_state= RANDOM_STATE, max_depth=rf_max_depth, n_estimators=rf_n_estimators), 
               LinearDiscriminantAnalysis(solver = lda_solver),
               MLPClassifier(random_state= RANDOM_STATE, hidden_layer_sizes=mlp_hidden_layer, activation=mlp_activation, solver=mlp_solver, max_iter=1000),
               svm.SVC(kernel='rbf', C=svc_c, gamma=svc_g),
               svm.SVC(kernel='sigmoid', C=svc_sig_c, gamma=svc_sig_g)]

In [66]:
pipeline_cross_val(names, Classifiers, embeddings_var, y_smk, kfold=kfold)

,Classifier,accuracy,balanced_accuracy,Precision,Recall,F1,AUC
0,K Nearest Neighbors,0.63 (0.05),0.63 (0.05),0.64 (0.06),0.60 (0.07),0.62 (0.06),0.66 (0.08)
1,Logistic Regression,0.68 (0.02),0.68 (0.02),0.70 (0.04),0.65 (0.05),0.67 (0.03),0.72 (0.01)
2,ExtraTrees Classifier,0.67 (0.02),0.67 (0.02),0.69 (0.02),0.64 (0.08),0.66 (0.04),0.71 (0.03)
3,Random Forest,0.70 (0.02),0.70 (0.02),0.71 (0.04),0.68 (0.08),0.69 (0.03),0.70 (0.02)
4,LDA,0.54 (0.05),0.54 (0.05),0.55 (0.06),0.53 (0.05),0.54 (0.04),0.59 (0.05)
5,MLP Classifier,0.67 (0.05),0.67 (0.05),0.67 (0.04),0.68 (0.09),0.67 (0.06),0.70 (0.05)
6,SVM rbf,0.66 (0.03),0.66 (0.04),0.68 (0.05),0.61 (0.04),0.64 (0.03),0.71 (0.02)
7,SVM sigmoid,0.67 (0.03),0.67 (0.03),0.69 (0.04),0.62 (0.06),0.65 (0.03),0.71 (0.01)


#### A-vowel phonation

In [51]:
embeddings_var = a_vowel_phonation_embeddings
embeddings_var.shape

(358, 2048)

##### Grid search with PCA

In [52]:
search_space = {'pca__n_components': [50, 100, 200, None],
                'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [3, 5, 7],
                 'classifier__weights': ['uniform', 'distance'],
                 }


pca_model =  Pipeline([('scaler', scaler), ('pca', PCA(random_state=42)), ('classifier', KNeighborsClassifier())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

knn_neighbours = clf.best_params_['classifier__n_neighbors']
knn_weights = clf.best_params_['classifier__weights']

print(clf.best_params_)

{'classifier': KNeighborsClassifier(n_neighbors=7), 'classifier__n_neighbors': 7, 'classifier__weights': 'uniform', 'pca__n_components': 50}


In [53]:
search_space = {'pca__n_components': [50, 100, 200, None],
                'classifier': [LogisticRegression(solver='liblinear')],
                'classifier__C': [0.01, 0.1, 1.0, 10]}

pca_model =  Pipeline([('scaler', scaler), ('pca', PCA(random_state=42)), ('classifier', LogisticRegression())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

lr_c = clf.best_params_['classifier__C']

print(print(clf.best_params_))

{'classifier': LogisticRegression(C=0.01, solver='liblinear'), 'classifier__C': 0.01, 'pca__n_components': 200}
None


In [54]:
search_space = {'pca__n_components': [50, 100, 200, None],
                'classifier': [ExtraTreesClassifier(random_state=RANDOM_STATE)],
                'classifier__n_estimators': [10, 30, 50, 100, 200],
                'classifier__max_depth': [8, 10, 12, 20]}

pca_model =  Pipeline([('scaler', scaler), ('pca', PCA(random_state=42)), ('classifier', ExtraTreesClassifier())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

etc_max_depth = clf.best_params_['classifier__max_depth']
etc_n_estimators = clf.best_params_['classifier__n_estimators']

print(clf.best_params_)

{'classifier': ExtraTreesClassifier(max_depth=8, n_estimators=200, random_state=42), 'classifier__max_depth': 8, 'classifier__n_estimators': 200, 'pca__n_components': 50}


In [55]:
search_space = {'pca__n_components': [50, 100, 200, None],
                'classifier': [RandomForestClassifier(max_depth=4, max_leaf_nodes=5, random_state=RANDOM_STATE)],
                'classifier__max_depth': [5, 10, None],
                'classifier__n_estimators': [10, 20, 30, 40, 50, 100]}

pca_model =  Pipeline([('scaler', scaler), ('pca', PCA(random_state=42)), ('classifier', RandomForestClassifier())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

rf_max_depth = clf.best_params_['classifier__max_depth']
rf_n_estimators = clf.best_params_['classifier__n_estimators']

print(clf.best_params_)

{'classifier': RandomForestClassifier(max_depth=5, max_leaf_nodes=5, n_estimators=50,
                       random_state=42), 'classifier__max_depth': 5, 'classifier__n_estimators': 50, 'pca__n_components': 100}


In [56]:
search_space = {'pca__n_components': [50, 100, 200, None],
                'classifier': [LinearDiscriminantAnalysis()],
                'classifier__solver': ['svd', 'lsqr']}

pca_model =  Pipeline([('scaler', scaler), ('pca', PCA(random_state=42)), ('classifier', LinearDiscriminantAnalysis())])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

lda_solver = clf.best_params_['classifier__solver']

print(clf.best_params_)


{'classifier': LinearDiscriminantAnalysis(), 'classifier__solver': 'svd', 'pca__n_components': 50}


In [57]:
search_space = {'pca__n_components': [50, 100, 200, None],
                'classifier': [MLPClassifier(random_state=RANDOM_STATE, max_iter = 1000)],
                 'classifier__hidden_layer_sizes': [(4,), (8,), (20,), (50,), (32,), (32,6,), (100,), (256, 128,)],
                 'classifier__activation': ['relu', 'tanh', 'logistic'],
                 'classifier__solver': ['adam', 'lbfgs']}

pca_model =  Pipeline([('scaler', scaler), ('pca', PCA(random_state=42)), ('classifier', MLPClassifier(random_state=1, max_iter=1000))])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

mlp_hidden_layer = clf.best_params_['classifier__hidden_layer_sizes']
mlp_activation = clf.best_params_['classifier__activation']
mlp_solver = clf.best_params_['classifier__solver']

print(clf.best_params_)

{'classifier': MLPClassifier(activation='tanh', hidden_layer_sizes=(32, 6), max_iter=1000,
              random_state=42), 'classifier__activation': 'tanh', 'classifier__hidden_layer_sizes': (32, 6), 'classifier__solver': 'adam', 'pca__n_components': None}


In [58]:
search_space = {'pca__n_components': [50, 100, 200, None],
                'classifier': [SVC(kernel='rbf')],
                'classifier__C': [1e4, 1e5, 1e6],
                'classifier__gamma': [1e-6, 1e-5, 1e-4, 1e-3]}

pca_model =  Pipeline([('scaler', scaler), ('pca', PCA(random_state=42)), ('classifier', SVC(kernel='rbf'))])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

svc_c = clf.best_params_['classifier__C']
svc_g = clf.best_params_['classifier__gamma']


print(clf.best_params_)

{'classifier': SVC(C=10000.0, gamma=0.0001), 'classifier__C': 10000.0, 'classifier__gamma': 0.0001, 'pca__n_components': None}


In [59]:
search_space = {'pca__n_components': [50, 100, 200, None],
                'classifier': [SVC(kernel='sigmoid')],
                'classifier__C': [1e4, 1e5, 1e6],
                'classifier__gamma': [1e-6, 1e-5, 1e-4, 1e-3]}

pca_model =  Pipeline([('scaler', scaler), ('pca', PCA(random_state=42)), ('classifier', SVC(kernel='sigmoid'))])

clf = GridSearchCV(pca_model, search_space, cv=kfold, verbose=False, scoring='accuracy', n_jobs=-1)
clf = clf.fit(embeddings_var, y_smk)

svc_sig_c = clf.best_params_['classifier__C']
svc_sig_g = clf.best_params_['classifier__gamma']

print(clf.best_params_)

{'classifier': SVC(C=10000.0, gamma=1e-06, kernel='sigmoid'), 'classifier__C': 10000.0, 'classifier__gamma': 1e-06, 'pca__n_components': None}


##### Cross-validation

In [60]:
# Model building to train
names = ['K Nearest Neighbors','Logistic Regression', 'ExtraTrees Classifier','Random Forest','LDA' ,'MLP Classifier' ,'SVM rbf', 'SVM sigmoid']
Classifiers = [
               KNeighborsClassifier(n_neighbors=knn_neighbours, weights=knn_weights),
               LogisticRegression(solver='liblinear', C=lr_c), 
               ExtraTreesClassifier(random_state= RANDOM_STATE, max_depth=etc_max_depth, n_estimators=etc_n_estimators),
               RandomForestClassifier(max_leaf_nodes=5, random_state= RANDOM_STATE, max_depth=rf_max_depth, n_estimators=rf_n_estimators), 
               LinearDiscriminantAnalysis(solver = lda_solver),
               MLPClassifier(random_state= RANDOM_STATE, hidden_layer_sizes=mlp_hidden_layer, activation=mlp_activation, solver=mlp_solver, max_iter=1000),
               svm.SVC(kernel='rbf', C=svc_c, gamma=svc_g),
               svm.SVC(kernel='sigmoid', C=svc_sig_c, gamma=svc_sig_g)]

In [61]:
pipeline_cross_val(names, Classifiers, embeddings_var, y_smk, reduction ='PCA', n_components=None, kfold=kfold)

,Classifier,accuracy,balanced_accuracy,Precision,Recall,F1,AUC
0,K Nearest Neighbors,0.56 (0.05),0.56 (0.05),0.56 (0.06),0.56 (0.10),0.56 (0.07),0.59 (0.06)
1,Logistic Regression,0.65 (0.05),0.65 (0.05),0.65 (0.04),0.63 (0.07),0.64 (0.06),0.69 (0.04)
2,ExtraTrees Classifier,0.59 (0.06),0.59 (0.06),0.59 (0.05),0.56 (0.06),0.57 (0.04),0.63 (0.06)
3,Random Forest,0.56 (0.06),0.56 (0.06),0.58 (0.08),0.44 (0.09),0.50 (0.07),0.57 (0.07)
4,LDA,0.44 (0.03),0.44 (0.03),0.49 (0.04),0.56 (0.08),0.52 (0.06),0.49 (0.07)
5,MLP Classifier,0.66 (0.05),0.66 (0.05),0.67 (0.05),0.63 (0.08),0.65 (0.06),0.70 (0.04)
6,SVM rbf,0.66 (0.06),0.66 (0.06),0.65 (0.05),0.68 (0.10),0.66 (0.07),0.68 (0.05)
7,SVM sigmoid,0.64 (0.05),0.64 (0.05),0.64 (0.04),0.64 (0.07),0.64 (0.05),0.66 (0.04)


In [62]:
pipeline_cross_val(names, Classifiers, embeddings_var, y_smk, reduction ='PCA', n_components=200, kfold=kfold)

,Classifier,accuracy,balanced_accuracy,Precision,Recall,F1,AUC
0,K Nearest Neighbors,0.54 (0.03),0.54 (0.03),0.54 (0.04),0.53 (0.09),0.53 (0.05),0.59 (0.06)
1,Logistic Regression,0.65 (0.04),0.65 (0.04),0.65 (0.04),0.64 (0.06),0.64 (0.05),0.68 (0.03)
2,ExtraTrees Classifier,0.59 (0.06),0.59 (0.06),0.59 (0.07),0.58 (0.08),0.58 (0.04),0.65 (0.05)
3,Random Forest,0.58 (0.07),0.58 (0.07),0.59 (0.08),0.49 (0.13),0.53 (0.10),0.60 (0.06)
4,LDA,0.60 (0.05),0.60 (0.05),0.61 (0.05),0.58 (0.08),0.59 (0.06),0.66 (0.04)
5,MLP Classifier,0.60 (0.03),0.60 (0.03),0.60 (0.03),0.59 (0.05),0.60 (0.03),0.67 (0.03)
6,SVM rbf,0.59 (0.03),0.59 (0.03),0.58 (0.04),0.66 (0.07),0.62 (0.04),0.68 (0.04)
7,SVM sigmoid,0.61 (0.04),0.61 (0.04),0.61 (0.04),0.62 (0.06),0.62 (0.05),0.67 (0.04)


In [63]:
pipeline_cross_val(names, Classifiers, embeddings_var, y_smk, reduction ='PCA', n_components=100, kfold=kfold)

,Classifier,accuracy,balanced_accuracy,Precision,Recall,F1,AUC
0,K Nearest Neighbors,0.59 (0.04),0.59 (0.04),0.59 (0.05),0.61 (0.08),0.60 (0.05),0.61 (0.04)
1,Logistic Regression,0.61 (0.03),0.61 (0.03),0.62 (0.03),0.59 (0.04),0.60 (0.03),0.67 (0.02)
2,ExtraTrees Classifier,0.63 (0.03),0.63 (0.03),0.65 (0.04),0.58 (0.08),0.60 (0.05),0.68 (0.05)
3,Random Forest,0.62 (0.01),0.62 (0.01),0.64 (0.02),0.55 (0.06),0.59 (0.03),0.63 (0.05)
4,LDA,0.61 (0.02),0.61 (0.02),0.61 (0.02),0.60 (0.04),0.60 (0.02),0.66 (0.03)
5,MLP Classifier,0.57 (0.05),0.57 (0.05),0.56 (0.05),0.61 (0.06),0.59 (0.06),0.64 (0.06)
6,SVM rbf,0.60 (0.01),0.60 (0.01),0.59 (0.02),0.68 (0.11),0.63 (0.04),0.67 (0.04)
7,SVM sigmoid,0.61 (0.02),0.61 (0.02),0.61 (0.01),0.61 (0.03),0.61 (0.02),0.65 (0.02)
